In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from utils import *

In [2]:

# maturity_c_dict = {'case1': [10, 0.15/2, 20],
#                   'case2': [5, 7.875/100/2, 10],
#                   'case3': [5, 7/100/4, 20],
#                   'case4': [7, 7.25/100/2, 14], # has government intervention
#                   'case5': [5, 4.22/100, 5]
#                   }




# data = pd.read_excel('../data/Charlie1124/'+ process_case + '/data-'+ process_case + '.xlsx')
# data = data.set_index('Date')
# date_flag = data.index.isin(pd.date_range(start='3/1/2023', end='3/17/2023'))
# data = data[date_flag]
# data = data.dropna(how = 'any')
# St = data['Stock'].values
# RET = data['return without dividend']/100

# r = data['r'].values/100
# coco_price = data['CoCo'].values
# T, c, M= maturity_c_dict[process_case]

# maturity = pd.to_datetime(data.index[0]) + pd.DateOffset(years = T)
# t0 = np.array(range(data.index.shape[0]))/252


# # [bond specifics]
# K = 100







In [ ]:
q = 0
K = 100
maturity_c_dict = {'case1': [10, 0.15/2, 20],
                  'case2': [5, 7.875/100/2, 10],
                  'case3': [5, 7/100/4, 20],
                  'case4': [7, 7.25/100/2, 14], # has government intervention
                  'case5': [5, 4.22/100, 5]
                  }
 
 
# [case1]
# process_case = 'case1'
# p = 0.7197
# w = 0.0435
# wbar = 1
# Jbar = 0.1811
 
# file_name = 'Lloyd (09-11).xlsx'
# stock_param = pd.read_csv('../param/J_' + file_name.split('.')[0] + 'casestudy.csv')
# loss, mu, sigma, l2, muV, sigmaV, eta, l1, b, a = stock_param.iloc[0].values
# k1, xi1, k2, xi2, l32, ignore_gov = [None, None, None, None, None, True]
 
 
# # [case4 17days]
#w:0.2347     p: 0.2844     Jbar: 0.4060    wbar: 0.3365 
process_case = 'case4'
 
# [parameters 2020]
p = 0.2844
w = 0.2347
wbar = 0.3365
Jbar = 0.4060
ignore_gov = False
pricing_period = ['1/1/2020', '3/17/2023']
 
# [parameters 2023, 17days]
# p = 0.5384
# w = 1.00
# wbar = 0.3068
# Jbar = 0.0
# ignore_gov = False
#pricing_period = ['3/1/2023', '3/17/2023']
 
file_name = 'Credit_Suisse_Data_13-23.xlsx'
intervention_param = pd.read_csv('../param/Intervention_' + file_name.split('.')[0] + '.csv')
(loss_a3, l1, b, a, mu_a3, sigma,
l2, muV, sigmaV, eta,
k1, xi1, k2, xi2, l32) = list(intervention_param.values.squeeze())
 
 
data = pd.read_excel('../data/Charlie1124/'+ process_case + '/data-'+ process_case + '.xlsx')
data = data.set_index('Date')
date_flag = data.index.isin(pd.date_range(start=pricing_period[0], end=pricing_period[1]))
data = data[date_flag]
 
data = data.dropna(how = 'any')
St = data['Stock'].values
r = data['r'].values/100
coco_price = data['CoCo'].values
T, c, M= maturity_c_dict[process_case]
maturity = pd.to_datetime(data.index[0]) + pd.DateOffset(years = T)
t0 = np.array(range(data.index.shape[0]))/252
 
 
model_price = equityconvert_coco(r, K, T, t0,
                                 l1, a, b,
                                 c, eta, p, q,
                                 Jbar, M, w, wbar,
                                 k1, xi1, k2, xi2,
                                 l2, l32, muV, sigmaV, sigma,
                                 ignore_gov=ignore_gov, St = St)
 
 
fig= plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.plot(data.index, coco_price, label = 'actual',  linewidth=2.0)
ax.plot(data.index, model_price,  label = 'estimated', linewidth=2.0)
ax.legend()
ax.set_ylabel('CoCo price')
ax.set_xlabel('Date')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
ax.set_title('Llyods Banking Group CoCo (11/23/2009 -- 12/30/2011)')
#ax.set_title('Credit Suisse CoCo (3/1/2023 -- 3/17/2023)')
#fig.savefig('../figure/llyods/llyods_casestudy.png', format='png', dpi=200)
plt.show()

In [ ]:
loss = np.sqrt(np.mean((model_price - coco_price) ** 2))
print(loss)